In [ ]:
Data_path="/content/drive/MyDrive/BarcodeData/partial_valid"

In [ ]:
from glob import glob
import xml.etree.ElementTree as ET
import os
import cv2
import numpy as np
from keras.applications import ResNet50
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.losses import Huber
from sklearn.model_selection import train_test_split

In [ ]:

images = []
annotations = []
original_images = []


image_paths = glob(os.path.join(Data_path, '*.jpg'))  

for img_path in image_paths:
    
    annotation_path = img_path.replace('.jpg', '.xml')


    image = cv2.imread(img_path)
    original_images.append(image)

  
    try:
        xml = ET.parse(annotation_path)
        root = xml.getroot()

      
        obj = root.find('object')
        if obj is not None:
            info = obj.find('bndbox')
            xmin = int(info.find('xmin').text)
            ymin = int(info.find('ymin').text)
            xmax = int(info.find('xmax').text)
            ymax = int(info.find('ymax').text)

          
            shape = image.shape
            xmin = xmin / shape[1]
            xmax = xmax / shape[1]
            ymin = ymin / shape[0]
            ymax = ymax / shape[0]

          
            image_resized = cv2.resize(image, (224, 224)) 

          
            images.append(image_resized)
            annotations.append((xmin, ymin, xmax, ymax))
        else:
            print(f"No object found in annotation for image: {img_path}")
    except FileNotFoundError:
        print(f"Annotation file not found for image: {img_path}")
    except ET.ParseError:
        print(f"Error parsing XML annotation for image: {img_path}")


images = np.array(images) / 255.0
annotations = np.array(annotations)

print(f"Loaded {len(images)} images and corresponding annotations.")



No object found in annotation for image: /content/drive/MyDrive/BarcodeData/partial_valid/939_qr_jpg.rf.744aa5044d66f87c2d71850c1088c528.jpg
No object found in annotation for image: /content/drive/MyDrive/BarcodeData/partial_valid/112_jpg.rf.820b5620604aca6f1984b154cc04572a.jpg
No object found in annotation for image: /content/drive/MyDrive/BarcodeData/partial_valid/1088_qr_jpg.rf.cb93485a9945d9b93cc899c926d7a668.jpg
No object found in annotation for image: /content/drive/MyDrive/BarcodeData/partial_valid/222_qr_jpg.rf.bbf2b92fd4110ba463374efc35ebc04c.jpg
No object found in annotation for image: /content/drive/MyDrive/BarcodeData/partial_valid/226_qr_jpg.rf.e6ae4a79643151bc09187735597949e8.jpg
No object found in annotation for image: /content/drive/MyDrive/BarcodeData/partial_valid/1093_qr_jpg.rf.be84c60052d8431db9b020d2d9a3ebbf.jpg
No object found in annotation for image: /content/drive/MyDrive/BarcodeData/partial_valid/807_qr_jpg.rf.d0972ef841b4914868f7a0220ce5ea2d.jpg
No object foun

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, annotations, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)
original_x_train, original_x_test, _, _ = train_test_split(original_images[:len(annotations)], annotations, test_size=0.2, random_state=42)
original_x_test, original_x_val, _, _ = train_test_split(original_x_test, original_x_test, test_size=0.5, random_state=42)

In [ ]:
model = Sequential()
model.add(ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)))
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(4, activation='sigmoid'))

model.layers[0].trainable = False

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 4)                   │           4,100 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 25,689,988 (98.00 MB)

 Trainable params: 2,102,276 (8.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
model.compile(optimizer='adam', loss=Huber(delta=1.0))
history = model.fit(np.array(x_train), np.array(y_train), validation_data=(np.array(x_val), np.array(y_val)), epochs=50)

Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 127s 7s/step - loss: 0.0383 - val_loss: 0.0185
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 132s 7s/step - loss: 0.0262 - val_loss: 0.0184
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 129s 6s/step - loss: 0.0230 - val_loss: 0.0074
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 142s 6s/step - loss: 0.0110 - val_loss: 0.0070
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 154s 7s/step - loss: 0.0101 - val_loss: 0.0065
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 131s 6s/step - loss: 0.0095 - val_loss: 0.0067
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 100s 6s/step - loss: 0.0084 - val_loss: 0.0066
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 139s 6s/step - loss: 0.0086 - val_loss: 0.0061
Epoch 9/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 152s 7s/step - loss: 0.0078 - val_loss: 0.0057
Epoch 10/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 133s 6s/step - loss: 0.0082 - val_loss: 0.0066
Epoch 11/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 149s 7s/step - loss: 0.0081 - val_loss: 0.0065
Epoch 12/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 143s 7s/step - loss: 0.0

In [ ]:
model.save('./model_50_resnet.keras')

In [ ]:
model = load_model('./model_50_resnet.keras')
y_pred = model.predict(x_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 16s 8s/step


In [ ]:
for i, a in enumerate(zip(y_pred, y_test)):
    x, y = a

    xmin, ymin, xmax, ymax = x[0], x[1], x[2], x[3]
    xmin, xmax = int(xmin * original_x_test[i].shape[1]), int(xmax * original_x_test[i].shape[1])
    ymin, ymax = int(ymin * original_x_test[i].shape[0]), int(ymax * original_x_test[i].shape[0])

    box = cv2.rectangle(original_x_test[i], (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

    cv2.imwrite(f'image_{i}.png', box)